In [ ]:
import pandas as pd
from dotenv import load_dotenv
import os

In [ ]:
cd ..

In [ ]:
from utils.capture_multiple_fred_series import capture_multiple_fred_series

In [ ]:
load_dotenv()  # take environment variables from .env.
fred_api_key = os.getenv('fred_api_key')

In [ ]:
combined_data = capture_multiple_fred_series(['FEDFUNDS','WILL5000IND'],fred_api_key)

In [ ]:
combined_data.dropna(how='all',inplace=True)

In [ ]:
combined_data.plot()

In [ ]:
fed_funds_rate = combined_data['fedfunds'].copy()

In [ ]:
fed_funds_rate = fed_funds_rate.resample('MS').first()
fed_funds_rate = fed_funds_rate.round(2)

In [ ]:
number_of_months = int(12*2.1)

fed_funds_rate_rolling_max = fed_funds_rate.rolling(number_of_months, min_periods=1).max()

fed_funds_rate_reversed = fed_funds_rate.iloc[::-1]
fed_funds_rate_rolling_max_forward_reversed = fed_funds_rate_reversed.rolling(number_of_months, min_periods=1).max()

fed_funds_rate_rolling_max_forward = fed_funds_rate_rolling_max_forward_reversed.iloc[::-1]

fed_funds_rate_peaks = fed_funds_rate[(fed_funds_rate == fed_funds_rate_rolling_max) & (fed_funds_rate == fed_funds_rate_rolling_max_forward)]

In [ ]:
fed_funds_rate_peaks = pd.DataFrame(fed_funds_rate_peaks)
fed_funds_rate_peaks.columns = ['peaks']
fed_funds_rate_peaks['peaks'] = fed_funds_rate_peaks['peaks'].astype('float64')

fed_funds_rate_peaks = fed_funds_rate_peaks[fed_funds_rate_peaks['peaks'].notnull()]
fed_funds_rate_peaks

In [ ]:
mask = pd.Series([False] * len(fed_funds_rate_peaks), index=fed_funds_rate_peaks.index)

for i in range(len(fed_funds_rate_peaks)):
    if i == 0 or fed_funds_rate_peaks['peaks'].iloc[i] != fed_funds_rate_peaks['peaks'].iloc[i-1]:
        mask.iloc[i] = True

result_df = fed_funds_rate_peaks[mask]

result_df

In [ ]:
# now that we have the peak dates, create a dataframe with a column for the date of the wilshire 5000 index value, the wilshire 5000 index value, the peak fed funds rate, and the peak fed funds rate date. 

wilshire_5000 = combined_data['will5000ind'].copy()

wilshire_5000 = wilshire_5000.resample('MS').first()

wilshire_5000 = wilshire_5000.round(2)

wilshire_5000 = pd.DataFrame(wilshire_5000)

wilshire_5000.columns = ['wilshire_5000']

wilshire_5000 = wilshire_5000[wilshire_5000['wilshire_5000'].notnull()]

wilshire_5000

In [ ]:
wilshire_5000 = wilshire_5000.merge(result_df, how='left', left_index=True, right_index=True)

wilshire_5000['peak_fed_funds_rate_date'] = wilshire_5000.index

wilshire_5000['peak_fed_funds_rate_date'] = wilshire_5000['peak_fed_funds_rate_date'].where(wilshire_5000['peaks'].notnull(), other=None)

wilshire_5000['peaks'] = wilshire_5000['peaks'].fillna(method='ffill', limit=24)

wilshire_5000['peak_fed_funds_rate_date'] = wilshire_5000['peak_fed_funds_rate_date'].fillna(method='ffill', limit=24)

wilshire_5000 = wilshire_5000[wilshire_5000['peaks'].notnull()]

wilshire_5000['wilshire_5000_pct_change'] = wilshire_5000.groupby('peak_fed_funds_rate_date')['wilshire_5000'].pct_change()+1

wilshire_5000['wilshire_5000_pct_change'] = wilshire_5000['wilshire_5000_pct_change'].fillna(1)

wilshire_5000

In [ ]:
wilshire_5000 = wilshire_5000[['peak_fed_funds_rate_date','wilshire_5000_pct_change']]
wilshire_5000

In [ ]:
wilshire_5000['rownumber'] = wilshire_5000.groupby('peak_fed_funds_rate_date').cumcount()
wilshire_5000

In [ ]:
wilshire_5000['cumulative_product'] = wilshire_5000.groupby('peak_fed_funds_rate_date')['wilshire_5000_pct_change'].cumprod()
wilshire_5000

In [ ]:
wilshire_5000 = wilshire_5000.pivot(index='rownumber', columns='peak_fed_funds_rate_date', values='cumulative_product')
wilshire_5000

In [ ]:
wilshire_5000.columns = pd.to_datetime(wilshire_5000.columns, format='%B %Y')
wilshire_5000 = wilshire_5000.reindex(sorted(wilshire_5000.columns), axis=1)
wilshire_5000

In [ ]:
wilshire_5000.columns = wilshire_5000.columns.strftime('%B %Y')
wilshire_5000

In [ ]:
wilshire_5000 = wilshire_5000 - 1
wilshire_5000 = wilshire_5000.round(4)
wilshire_5000 = wilshire_5000.rename(columns={'rownumber':'months_since_peak'})
wilshire_5000

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick

wilshire_5000.plot(colormap='Blues', legend=False)

plt.title('Wilshire 5000 Index Performance Relative to Peak Fed Funds Rate')
plt.xlabel('Months Since Peak Fed Funds Rate')
plt.ylabel('Wilshire 5000 Index Performance')
plt.legend(loc='upper left')

fmt = '%.1f%%'
yticks = mtick.FormatStrFormatter(fmt)
plt.gca().yaxis.set_major_formatter(yticks)

plt.show()

In [ ]:
index_return_data = combined_data['will5000ind'].copy()

index_return_data = index_return_data.resample('MS').first()

index_return_data.dropna(inplace=True)
index_return_data = pd.DataFrame(index_return_data)
index_return_data.reset_index(inplace=True)
index_return_data

In [ ]:
new_dataframe = pd.DataFrame()

for i in range(len(index_return_data)):
    if i + 24 < len(index_return_data):
        temp_dataframe = index_return_data.iloc[i:i+25].copy()
        temp_dataframe['beginning_date'] = index_return_data['date'].iloc[i]
        temp_dataframe['beginning_value'] = index_return_data['will5000ind'].iloc[i]
        new_dataframe = pd.concat([new_dataframe, temp_dataframe], ignore_index=True)

new_dataframe['row_number'] = new_dataframe.groupby('beginning_date')['date'].cumcount()

new_dataframe['cumulative_product'] = new_dataframe['will5000ind']/new_dataframe['beginning_value']

new_dataframe

In [ ]:
average_cumulative_product = new_dataframe.groupby('row_number')['cumulative_product'].mean()-1
average_cumulative_product


In [ ]:
combined_df = pd.concat([wilshire_5000, average_cumulative_product], axis=1)
combined_df.rename(columns={'cumulative_product':'Mean cumulative return over period'}, inplace=True)
combined_df